In [20]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

In [21]:
data = pd.read_csv('data_cleaned.csv')
data.drop(['ESTU_GENERO'], axis=1, inplace=True)
data 

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_TIENECOMPUTADOR,FAMI_TIENELAVADORA,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENESERVICIOTV,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_CUANTOSCOMPARTEBAÑO,ESTU_HORASSEMANATRABAJA,PUNT_GLOBAL
0,6.25,0,6,8,2,1,0,1,0,1,0,0,0,3.5,15.5,165
1,6.25,1,5,6,3,1,0,1,1,0,0,0,0,3.5,25.5,138
2,4.75,0,4,6,2,0,1,1,0,1,0,0,0,3.5,15.5,129
3,4.75,0,1,6,1,1,1,1,0,1,0,0,0,3.5,10.0,164
4,3.25,1,6,7,3,1,1,1,0,1,1,1,0,2.0,15.5,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37813,0.00,0,1,1,3,1,1,1,1,1,0,1,1,3.5,10.0,121
37814,0.00,0,9,0,3,1,1,1,0,1,1,0,0,3.5,30.0,135
37815,6.25,0,6,9,3,1,1,1,1,1,1,0,0,3.5,30.0,129
37816,0.00,1,0,1,1,1,1,1,0,0,0,1,0,3.5,30.0,125


In [22]:
features = data.drop(columns=['PUNT_GLOBAL'])
tarjet = data['PUNT_GLOBAL']

scaler = preprocessing.StandardScaler()
features = preprocessing.normalize(features)
features = scaler.fit_transform(features)

pca = PCA(n_components=1)
features_pca = pca.fit_transform(features)

features_train, features_test, tarjet_train, tarjet_test = train_test_split(features_pca, tarjet, test_size=0.2, random_state=0)


In [23]:
kmeans = KMeans(n_clusters=2, random_state=0, init='k-means++', n_init=100)
clusters = kmeans.fit_predict(features_train)

In [24]:
score = silhouette_score(features_train, clusters)
print(score, '%')

0.7071357838436816 %


In [25]:
type(kmeans.cluster_centers_)

numpy.ndarray

In [26]:
new_df = pd.DataFrame(features_train, columns=['PCA_1'])
new_df['Cluster'] = clusters
new_df['Cluster'].replace({0: 'Cluster 1', 1: 'Cluster 2'}, inplace=True)
new_df

/tmp/ipykernel_276326/3188319708.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_df['Cluster'].replace({0: 'Cluster 1', 1: 'Cluster 2'}, inplace=True)


,PCA_1,Cluster
0,2.977734,Cluster 1
1,1.928369,Cluster 1
2,1.984079,Cluster 1
3,1.251088,Cluster 1
4,2.201707,Cluster 1
...,...,...
30249,2.652083,Cluster 1
30250,0.539259,Cluster 1
30251,-0.273429,Cluster 1
30252,1.227759,Cluster 1


In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
new_tarjet = new_df['Cluster']
new_features = new_df.drop(columns=['Cluster'])
features_train, features_test, tarjet_train, tarjet_test = train_test_split(new_features, new_tarjet, test_size=0.2, random_state=1)
knn = KNeighborsClassifier(n_neighbors=40, metric='euclidean')
knn.fit(features_train, tarjet_train)
predictions = knn.predict(features_test)
result = knn.score(features_test, tarjet_test)
accuracy = metrics.accuracy_score(tarjet_test, predictions)*100
print(int(accuracy),'%')

99 %
